In [2]:
#Imports
%load_ext Cython
import random
from random import randrange
import math
import copy
import numpy as np
from random import shuffle
import sys
import timeit

In [3]:
%%cython
cimport numpy
cpdef cysum(numpy.ndarray[double] A,numpy.ndarray[double] B):
    """Compute the sum of an array"""
    cdef int count=0
    smax = B.shape[0]
    
    for i in range(A.shape[0]):
        for j in range(i,A.shape[0]):
            if(A[i]==1 and A[j]==1):
                B[count] = 1
            else:
                B[count] = 0    
            count = count + 1
    return B
cpdef threecysum(numpy.ndarray[double] A,numpy.ndarray[double] B):
    """Compute the sum of an array"""
    cdef int count=0
    smax = B.shape[0]
    for i in range(A.shape[0]):
        for j in range(i,A.shape[0]):
            for k in range(j,A.shape[0]):
                if((A[i]==1 and A[j]==1) or(A[i]==1 and A[j]==1 and A[k]==1)):
                    B[count] = 1
                else:
                    B[count] = 0    
                count = count + 1
    return B

#combina 2 vetores num 3 vetor
cpdef cyComb(numpy.ndarray[double] A,numpy.ndarray[double] B,numpy.ndarray[double] C,int x):    
    for i in range(A.shape[0]):
        for j in range(B.shape[0]):
            if(A[i]==1 and B[j]==1):
                C[x] = 1
            x+=1
    return [C,x]


In [4]:
#Logica do jogo
class Domino:  
    def __init__(self,winReward,lostReward,blockingReward,blockedReward,lostPunition,gameType):
        self.winReward = winReward
        self.lostReward = lostReward
        self.blockingReward = blockingReward
        self.blockedReward = blockedReward
        self.lostPunition = lostPunition
        self.history = []
        self.gameType = gameType
        self.drawing = False
        
        
    def buy(self,hand, n, pieces):
        i = 0
        while i<n:
            hand.append(pieces[i])
            i=i+1
        del pieces[0:n]
        return [hand,pieces]

    #recebe estado do jogo, retorna ações possíveis
    def possibleActions(self,state,player):
        status = state[0]
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
        l_end = state[4]
        r_end = state[5]
        boneyard = state[8]
        actions = []
        index = -1

        if(l_end==r_end==-1):
            for piece in hand:
                index +=1
                actions.append((piece[0],piece[1],index,"hand"))
            return actions
        
        #para o game de block e draw
        actions = self.checkMatch(hand,actions,l_end,r_end,"hand")
        '''
        pos 2 e 3 no actions se referem a nova ponta na mesa:
        exemplo: campo 1-5 , mão : (1,2),(5,5)
        nesse caso a peça (1,2) se encaixa no lado esquerdo, sendo
        a nova ponta esquerda o 2. A representação será: (0,1), pois
        0 é a posição da tupla na mão, e o 1 representa a posição na tupla,
        que no caso é o direito

        '''
        #para o game de draw somente, caso tenha que comprar pecas
        if(not actions and self.gameType == "Draw" ):
            #print "lol"
            #print actions
            #print state
            boneyard = state[8]
            if(len(boneyard)>0):
                actions = self.checkMatch(boneyard,actions,l_end,r_end,"boneyard") #busca no boneyard
        if(not actions):
            actions.append(None)
        return actions
    #action(valor significante,lado significante, posicao na hand ou boneyard, hand ou boneyard(se veio))
    def checkMatch(self,pieces,actions,l_end,r_end,source):
        index = -1
        for piece in pieces:
            index +=1
            #peça dupla
            if(piece[0]==piece[1]):
                if(piece[0]==l_end):
                    actions.append((piece[1],"left",index,source))
                elif(piece[1]==r_end):
                    actions.append((piece[0],"right",index,source))
                continue
            if(piece[0]==l_end):
                actions.append((piece[1],"left",index,source))
                if(l_end==r_end): #evitar duplicacao de mesmas ações na esq e dir
                    continue
            if(piece[1]==l_end):
                actions.append((piece[0],"left",index,source))
                if(l_end==r_end):
                    continue
            if(piece[0]==r_end):
                actions.append((piece[1],"right",index,source))
                if(l_end==r_end):
                    continue
            if(piece[1]==r_end):
                actions.append((piece[0],"right",index,source))
                if(l_end==r_end):
                    continue
        return actions

    def termination(self,state):#se a partida ja terminou
        status = state[0]
        if (status >= 3):
            return True  
        return False
    
    def reward(self,state,player):#recompensa por estar nesse estado
        #em caso de vitoria
        status = state[0]
        p1_hand = state[1]
        p2_hand = state[2]
        l_end = state[4]
        r_end = state[5]
        if(status == 1):
            return 0
        
        if((state[6] != state[7] ) and status < 3):#avaliar block    
            if(state[6]):
                if(player == "p1"):#fui bloqueado
                    return self.blockedReward
                else:
                    return self.blockingReward
            else:
                if(player == "p2"):#fui bloqueado
                    return self.blockedReward 
                else:
                    return self.blockingReward
            
        if(status >= 3):
            p1_hand = state[1]
            p2_hand = state[2]
            lighestP1Value = 13
            lighestP2Value = 13
            lighestP1Tile = 13
            lighestP2Tile = 13
            p1_total_value = 0
            p2_total_value = 0
            for piece in p1_hand:
                value = piece[0]+piece[1]
                p1_total_value+=value
            for piece in p2_hand:
                value = piece[0]+piece[1]
                p2_total_value+=value             
                
            if(p1_total_value<p2_total_value):
                state[0] = 4#player 1 venceu tendo a menor mao
                if(player == "p1"):
                    return self.winReward + p2_total_value
                elif(self.lostPunition):# se é pra punir pq perdeu
                    return self.lostReward - p2_total_value # punicao mais pontos que tinha
                else:
                    return 0
                
            elif(p1_total_value>p2_total_value):
                state[0] = 5#player 2 venceu tendo a menor mao
                if(player == "p2"):
                    return self.winReward + p1_total_value
                elif(self.lostPunition):
                    return self.lostReward - p1_total_value
                else:
                    return 0
            else:#empate
                for piece in p1_hand:
                    if(piece[0]<lighestP1Value):
                        lighestP1Value = piece[0]
                        lighestP1Tile = piece[0]+piece[1]
                    if(piece[1]<lighestP1Value):
                        lighestP1Value = piece[1]
                        lighestP1Tile = piece[0]+piece[1]
                        
                for piece in p2_hand:
                    if(piece[0]<lighestP2Value):
                        lighestP2Value = piece[0]
                        lighestP2Tile = piece[0]+piece[1]
                    if(piece[1]<lighestP2Value):
                        lighestP2Value = piece[1]
                        lighestP2Tile = piece[0]+piece[1]
                        
                if(lighestP1Value<lighestP2Value):
                    state[0] = 4#player 1 venceu tendo o menor valor entre o par das pecas    
                    if(player == "p1"):
                        return self.winReward + p2_total_value
                    elif(self.lostPunition):
                        return self.lostReward - p2_total_value
                    else:
                        return 0
                elif(lighestP1Value>lighestP2Value):
                    state[0] = 5#player 2 venceu tendo o menor valor entre o par das pecas
                    if(player == "p2"):
                        return self.winReward + p1_total_value 
                    elif(self.lostPunition):
                        return self.lostReward - p1_total_value
                    else:
                        return 0
                else:
                    if(lighestP1Tile<lighestP2Tile):#menor valor da peça como um todo
                        state[0] = 4
                        if(player == "p1"):
                            return self.winReward + p2_total_value
                        elif(self.lostPunition):
                            return self.lostReward - p2_total_value
                        else:
                            return 0
                    else:#nao pode haver empate
                        state[0] = 5
                        if(player == "p2"):
                            return self.winReward + p1_total_value 
                        elif(self.lostPunition):
                            return self.lostReward - p1_total_value
                        else:
                            return 0

    def undoHistory(self,state,player):#ele considera que o player foi o ult a jogar
        if(len(self.history)>0):
            action = self.history.pop()
            if(action is None or action[0] is None):
                if(state[0]==3):#N DEVE existir um estado de vitoria anterior a este
                    state[0] = 1
                    return state #ele sempre tera none antes da vitoria 
                if(state[0]==4 or state[0]==5):#caso ele avalie uma 2 vez dps do 3
                    return state #ele n deve dar undo apos vitoria 
                if(len(self.history)>0):#
                    action2 = self.history.pop()
                    if(action2 is not None):#se ja n estava bloqueado
                        if(player == "p1"):
                            state[6] = False
                        else:
                            state[7] = False
                    self.history.append(action2)
                else:
                    if(player == "p1"):#ninguem começa bloqueado
                        state[6] = False
                    else:
                        state[7] = False
                
                    
                return state     #o alg se encarrega de dar undo corretamente      
                
            field = state[3]
            l_end = state[4]
            r_end = state[5]
            
            #print "Action taken out: "+str(action)
            #print self.history
            #print state
                
            piece = field.pop()#nunca deve estar vazio,se estiver o problema e a historia
            
            if(state[0]==3):#TOMAR MUITO CUIDADO AKI
                state[0] = 1
            
            if(player == "p1"):
                hand = state[1]
            else:
                hand = state[2]
            #print action
            act = action[0]
            pos = act[2]
            if(len(self.history)>0):#se nao foi a 1 jogada
                if(piece[0]!=piece[1]):#se n for peça dupla
                    if(piece[0] == act[0]):
                        if(act[1]=="left"):
                            l_end = piece[1] #só pode ter o msm valor que o lado dela
                        else:
                            r_end = piece[1]
                    elif(piece[1] == act[0]):
                        if(act[1]=="left"):
                            l_end = piece[0] #só pode ter o msm valor que o lado dela
                        else:
                            r_end = piece[0]        
                else:
                    if(act[1]=="left"):
                        l_end = piece[1] #tomar cuidado aki, pra n conflitar com
                    else:
                        r_end = piece[1]# o tratamento a peças duplas na ação
                hand.insert(pos,piece)
                
            else:
                l_end = r_end = -1
                hand.insert(pos,piece)
            
            if(player == "p1"):
                state[1] = hand
            else:
                state[2] = hand
                
            state[3] = field
            state[4] = l_end
            state[5] = r_end
            return state
        else:
            if(state[0]==3):#TOMAR MUITO CUIDADO AKI
                state[0] = 1
            return state
            
            
            
            
            
            
            
            
            
    
    def startGame(self):
        status = 1 #1=in progress; 2=one player blocked;3=two players blocked;4/5=p1 won/p2 won
        boneyard = [(x,y) for x in range(7) for y in range(x,7)]
        random.shuffle(boneyard)
        p1_hand,boneyard = self.buy([],7,boneyard)
        p2_hand,boneyard = self.buy([],7,boneyard)
        field = []
        l_end = -1
        r_end = -1
        player1Blocked = False
        player2Blocked = False
        self.history = []  
        turn = 0 #turno de jogo que vai incrementando a cada jogada
        #p2 hand n e visivel, mas o agente imperfeito pode utilizar len() para saber o 
        #tamanho da mao do oponente
        state = [status, p1_hand, p2_hand, field,l_end,r_end,player1Blocked,player2Blocked,boneyard,turn]
        return state

    def playAction(self,state,action,player):
        status = state[0]
        if(status >= 3): #jogo acabou
            return state
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
            
        field = state[3]
        l_end = state[4]
        r_end = state[5]
        boneyard = state[8]

        if(self.gameType=="Draw"):
            if(action[0] is None):#nenhuma peca no boneyard e jogavel
                if(player =="p1"):
                    state[1]+=state[8] #receba todas as pecas do boneyard
                    state[8] = []
                    state[6] = True
                elif(player =="p2"):
                    state[2]+=state[8]
                    state[8] = []
                    state[7] = True
                if(state[6] and state[7]): #ambos estao bloqueados
                    state[0] = 3
                return state
            
            else: 
                if(player =="p1"):
                    state[6] = False #se estava em block, nao esta mais
                else:
                    state[7] = False
                p_index = action[2]  

                if(l_end==r_end==-1):
                    orientation = "left"
                else:
                    orientation = action[1]
                if(action[3]=="boneyard"):#se ele esta jogando peca do boneyard
                    #print "action: " + str(action)
                    #print : " + str(boneyard)
                    p = boneyard[p_index]
                    field.append(p)
                    boneyard.remove(p) 
                else: 
                    #print "action: " + str(action)
                    #print "hand: " + str(hand)
                    p = hand[p_index]
                    field.append(p)
                    hand.remove(p)
                
                self.drawing = False
                
        elif(self.gameType=="Block"):
            
            if(action is None):#foi bloqueado
                self.history.append(None)
                if(player =="p1"):
                    state[6] = True
                else:
                    state[7] = True
                if(state[6] and state[7]): #ambos estao bloqueados
                    state[0] = 3
                return state

            if(action[0] is None):#foi bloqueado no block
                self.history.append([action[0]])
                if(player =="p1"):
                    state[6] = True
                else:
                    state[7] = True
                if(state[6] and state[7]):
                    state[0] = 3
                return state 
        
            #dado que o block acima n ocorreu, entao temos peca para nos livrar de um eventual block
            if(player =="p1"):
                state[6] = False
            else:
                state[7] = False
            p_index = action[2]  

            if(l_end==r_end==-1):
                orientation = "left"
            else:
                orientation = action[1]
            
            p = hand[p_index]
            self.history.append([action])#action ja vem com a pos atualizada
            field.append(p)
            hand.remove(p)
    
        if(l_end==-1 and r_end==-1):
            l_end, r_end = p
        elif(orientation == "right"):#ori e o lado desejado a manter na ponta
            r_end=action[0]
        elif(orientation == "left"):
            l_end=action[0]
                
        if(player =="p1"):
            state[1] = hand
        else:
            state[2] = hand
            
        if(len(hand) ==0): #zerou a mao, acabou o jogo
            state[0] = 3
            
        state[3] = field
        state[4] = l_end
        state[5] = r_end
        state[9] +=1 #incrementa 1 turno
        return state#mecanica do jogo

In [ ]:
#funcoes auxiliares utilizadas pelo validator
class Rules:
    
    def isTurn(self,turn,exp):
        if(exp(turn)):
            return True
        return False
    
    def NumDouble(self,hand,action,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(piece[0] == piece[1]):
                count +=1
        if(exp(count)):
            return True
        return False
    
    def HandSize(self,hand,exp):
        lenght = len(hand)
        return exp(lenght)
    
    def hasDouble(self,number,hand):
        for piece in hand:
            if((piece[0] == piece[1]) and (piece[0] == number)):
                return True
        return False
    
    def hasPiece(self,hand,myPiece):
        for piece in hand:
            if( myPiece == piece ):
                return True
        return False
    
    def hasPieceValue(self,hand,exp):
        for piece in hand:
            value = piece[0] + piece[1]
            if(exp(value)):
                return True
        return False
    
    def hasManyPieceValues(self,hand,exp):
        count = 0
        for piece in hand:
            value = piece[0] + piece[1]
            if(exp(value)):
                count+=1
        return count
    
    def hasHandValue(self,hand):
        value = 0
        for piece in hand:
            value += piece[0] + piece[1]
        return value
    
    def blocks(self,hand,l_end,r_end):
        if(l_end==-1 or r_end==-1):
            return False
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                #se encontra alguma peça valida na mao
                return False # então ainda não estou bloqueado
        return True #nao encontrou
        
         

    def NumActions(self,hand,l_end,r_end):
        count = 0
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                count +=1
            if(self.specialCase(piece,l_end,r_end)):
                count+=1 
               
        return count
    
    def shortPiece(self,number,handOp):
        for piece in handOp:
            if(piece[0]==number or piece[1]==number):
                return False
        return True #op n tem nenhuma peça dese tipo
        
          
    
    def hasSpecialCase(self,hand,l_end,r_end):
        for piece in hand:
            if((piece[0]==l_end and piece[1]==r_end) or (piece[0]==r_end and piece[1]==l_end)):
                if(l_end!=r_end):
                    return True
        return False
    
    def isPlayable(self,piece,l_end,r_end):#se dado a peça e as duas pontas, ela é jogavel
        if(piece[0]==l_end or piece[0]==r_end or piece[1]==l_end or piece[1]==r_end):
            return True
        if(l_end == r_end == -1):
            return True
        return False   
    
    #tratar caso especial. Ex:
    # l = 5, r = 4 , piece = (5,4) , acoes possiveis sao 2: (4,l) e (5,r)
    def specialCase(self,piece,l_end,r_end):
        if((piece[0]==l_end and piece[1]==r_end) or (piece[0]==r_end and piece[1]==l_end)):
                if(l_end!=r_end):
                    return True
        return False

In [6]:
#funcoes auxiliares utilizadas pelo validator
class Rules2:
    
    def isTurn(self,turn,exp):
        if(exp(turn)):
            return True
        return False
    
    def NumDouble(self,hand,action,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(piece[0] == piece[1]):
                count +=1
        if(exp(count)):
            return True
        return False
    
    def HandSize(self,hand,exp):
        lenght = len(hand)
        return exp(lenght)
    
    def hasDouble(self,number,hand):
        for piece in hand:
            if((piece[0] == piece[1]) and (piece[0] == number)):
                return True
        return False
    
    def hasPiece(self,hand,myPiece):
        for piece in hand:
            if( myPiece == piece ):
                return True
        return False
    
    def hasPieceValue(self,hand,exp):
        for piece in hand:
            value = piece[0] + piece[1]
            if(exp(value)):
                return True
        return False
    
    def hasManyPieceValues(self,hand,exp):
        count = 0
        for piece in hand:
            value = piece[0] + piece[1]
            if(exp(value)):
                count+=1
        return count
    
    def hasHandValue(self,hand,exp):
        value = 0
        for piece in hand:
            value += piece[0] + piece[1]
        if(exp(value)):
            return True
        else:
            return False
    
    def blocks(self,hand,l_end,r_end):
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                #se encontra alguma peça valida na mao
                return False # então ainda não estou bloqueado
        return True #nao encontrou
        
    def NumActions(self,hand,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                count +=1
        if(exp(count)):#se tem mais que o numero de peças designado
            return True
        return False #se tem exatamente aquele numero
    
    def shortPiece(self,number,handOp):
        for piece in handOp:
            if(piece[0]==number or piece[1]==number):
                return False
        return True #op n tem nenhuma peça dese tipo
        
          
    def isPlayable(self,piece,l_end,r_end):#se dado a peça e as duas pontas, ela é jogavel
        if(piece[0]==l_end or piece[0]==r_end or piece[1]==l_end or piece[1]==r_end):
            return True
        return False   

In [7]:
#RN 35 caracteristicas
class Network:
    def validate(self,result,hand,handOp,action,player,state):
        rules = Rules2()
        l_end = state[4]
        r_end = state[5]
        """
        Características a partir de informações imperfeitas.
            Somente do que é visível a um jogador comum:
                Campo, peças de suas mãos, quantidade de peças do oponente
        """
        #valida caracteristicas de acordo com o estado submetido e as retorna

        result[0] = rules.NumDouble(hand,action,l_end,r_end,lambda x: x==1)
        result[1] = rules.NumDouble(hand,action,l_end,r_end,lambda x: x==2)
        result[2] = rules.NumDouble(hand,action,l_end,r_end,lambda x: x>=3)
        #impossibilita qualquer peça de ser jogada
        result[3] = rules.blocks(hand,l_end,r_end)
        
        result[4] = rules.NumActions(hand,l_end,r_end,lambda x: x==1)
        result[5] = rules.NumActions(hand,l_end,r_end,lambda x: x==2)
        result[6] = rules.NumActions(hand,l_end,r_end,lambda x: x>=3)
       
        for i in range(7,14):
            #quais peças op tem na mão, de 0 a 6
            result[i] = rules.hasDouble(i-7,hand)
                
        """
        Perfect Information
        """
        result[14] = rules.blocks(handOp,l_end,r_end)
        result[15] = rules.NumActions(handOp,l_end,r_end,lambda x: x==1)
        result[16] = rules.NumActions(handOp,l_end,r_end,lambda x: x==2)
        result[17] = rules.NumActions(handOp,l_end,r_end,lambda x: x>=3)
        for i in range(18,25):
            #quais peças duplas temos na mão, de 0:0 a 6:6
            result[i] = rules.shortPiece(i-18,handOp)
        
        
        result[25] = rules.hasPieceValue(hand,lambda x: x>=9)
        result[26] = rules.hasPieceValue(hand,lambda x: x<=3)
        result[27] = rules.hasHandValue(hand,lambda x: x<=4)
        result[28] = rules.hasHandValue(hand,lambda x: x>=12 and x<=19)
        result[29] = rules.hasHandValue(hand,lambda x: x>=20)
            
        result[30] = rules.hasPieceValue(handOp,lambda x: x>=9)
        result[31] = rules.hasPieceValue(handOp,lambda x: x<=3)
        result[32] = rules.hasHandValue(handOp,lambda x: x<=4)
        result[33] = rules.hasHandValue(handOp,lambda x: x>=12 and x<=19)
        result[34] = rules.hasHandValue(handOp,lambda x: x>=20)
                    
        #result[35] = rules.HandSize(hand,lambda x: x<=3)
        #result[36] = rules.HandSize(hand,lambda x: x>3)
        #result[37] = rules.HandSize(handOp,lambda x: x<=3)
        #result[38] = rules.HandSize(handOp,lambda x: x>3)
        return result  

In [8]:
#RN Muitas características
class ExtensiveNetwork:
     def validate(self,result,hand,handOp,action,player,state):
        rules = Rules()
        l_end = state[4]
        r_end = state[5]
        turn = state[9]
        
        #result = np.zeros(110)
        result[0] = rules.blocks(hand,l_end,r_end)
        
        for i in range(1,8):
            #quais peças op tem na mão, de 0 a 6
            result[i] = rules.hasDouble(i-1,hand)
            
        result[8] = rules.blocks(handOp,l_end,r_end)
        
        myHandValue = rules.hasHandValue(hand)
        result[9] = myHandValue<7
        result[10] = myHandValue>=7 and myHandValue<=15
        result[11] = myHandValue>15 and myHandValue<=25
        result[12] = myHandValue>25 and myHandValue<=35
        result[13] = myHandValue>35 and myHandValue<=47
        result[14] = myHandValue>47
        
        opHandValue = rules.hasHandValue(handOp)
        result[15] = opHandValue<7
        result[16] = opHandValue>=7 and opHandValue<=15
        result[17] = opHandValue>15 and opHandValue<=25
        result[18] = opHandValue>25 and opHandValue<=35
        result[19] = opHandValue>35 and opHandValue<=47
        result[20] = myHandValue>47
        #Esses valores altos(35 pra cima, sao a media pra uma mao com 7)
        result[21] = rules.hasSpecialCase(hand,l_end,r_end)
        result[22] = rules.hasSpecialCase(handOp,l_end,r_end)
        j = 23
        counter = 0
        numAct = rules.NumActions(hand,l_end,r_end)
        for i in range(j,j+6):
            result[i] = numAct == counter
            counter +=1
        j+=counter
        result[j] = numAct >= 6
        j+=1
        
        counter = 0
        numAct = rules.NumActions(hand,l_end,r_end)
        for i in range(j,j+6):
            result[i] = numAct == counter
            counter +=1
        j+=counter
        result[j] = numAct >= 6
        j+=1
        #turnos
        for i in range(13):
            result[j] = rules.isTurn(turn,lambda x: x==i)
            j+=1
        result[j]  = rules.isTurn(turn,lambda x: x>13)
        j+=1
        manyPieceArray = np.zeros(28)
        #count e a contagem de pecas diferentes com a soma dos valores respeitado a funcao
        count1 = rules.hasManyPieceValues(hand,lambda x: x<3)
        count2 = rules.hasManyPieceValues(hand,lambda x: x>=3 and x<=6)
        count3 = rules.hasManyPieceValues(hand,lambda x: x>6 and x<=9)
        count4 = rules.hasManyPieceValues(hand,lambda x: x>9)
        
        for i in range(0,7):
            result[j] = i == count1#de 1-6 pecas diferentes na mao com valor menor q 3
            result[j+7] = i == count2#de 1-6 pecas diferentes na mao com valor entre 3 e 6
            result[j+14] = i == count3#de 1-6 pecas diferentes na mao com valor entre 7 e 9
            result[j+21] = i == count4#de 1-6 pecas diferentes na mao com valor maior que 9
            j+=1
        j += 21
        manyPieceArray = np.zeros(28)
        #count e a contagem de pecas diferentes com a soma dos valores respeitado a funcao
        count1 = rules.hasManyPieceValues(handOp,lambda x: x<3)
        count2 = rules.hasManyPieceValues(handOp,lambda x: x>=3 and x<=6)
        count3 = rules.hasManyPieceValues(handOp,lambda x: x>6 and x<=9)
        count4 = rules.hasManyPieceValues(handOp,lambda x: x>9)
        
        for i in range(0,7):
            result[j] = i == count1#de 1-6 pecas diferentes na mao com valor menor q 3
            result[j+7] = i == count2#de 1-6 pecas diferentes na mao com valor entre 3 e 6
            result[j+14] = i == count3#de 1-6 pecas diferentes na mao com valor entre 7 e 9
            result[j+21] = i == count4#de 1-6 pecas diferentes na mao com valor maior que 9
            j+=1
        
        j += 21
        #threewise_result = threecysum(result,threewise_result)
        #if(result[37]==1):
            #print player
            #print state 
        
        return result

In [9]:
#Classe principal que computa as caracteristicas presentes numa RN
class Features:
    def __init__(self,featureWeights,featureValidator):
        self.data = []
        self.featureWeights = featureWeights
        self.numFeatures = featureWeights.size
        self.featureValidator = featureValidator
           
    def getFeaturesWeights(self):
        return self.featureWeights
    
    def setFeaturesWeights(self,featureWeights):
        self.featureWeights = featureWeights
        
    def atomicFeatures(self,state,action,player):
        result =  np.zeros(self.numFeatures)
        
        #Trate como afterState
        l_end = state[4]
        r_end = state[5]
        if(player == "p1"):
            hand = state[1]
            handOp = state[2]
        elif(player == "p2"):
            hand = state[2]
            handOp = state[1]
        if(action is not None):
            if(action[0]!=None):#quando nao for bloqueado    
                pos = action[2]
                if(l_end==r_end==-1):
                    pos = action[2]
                    l_end = action[0]
                    r_end = action[1]
                elif(action[1]=="left"):
                #substitui uma das pontas pela nova ação
                    l_end = action[0]
                else:
                    r_end = action[0]
                #print "pos: "+str(pos)
                value =hand.pop(pos)
            
        #featuresValidator = FeaturesValidator()
        result = self.featureValidator.validate(result,hand,handOp,action,
                                                player,state)
        if(action is not None):
            if(action[0]!=None):
                hand.insert(pos,value)
        
        return result #funcao principal que processa a feature 

In [10]:
class Node:
    def __init__(self,player,act):
        self.player = player
        self.act = act
        self.children = []
        self.max_child_number = 0
        self.know_children_number = 0
        self.valueQ = 0.0
        self.total = 0.0
        self.count = 0
        self.identifier = 0
        self.firstTime = True
        self.expanded = False
        self.isTerminal = False
        self.reward = 0 #recompensa utilizada quando esse for o no terminal
    def choosen_act(self,actions):
        try:
            return actions[self.know_children_number]
        except:
            print "Unexpected error:", sys.exc_info()[1]
            print "error on choosen act, children number:" + str(self.know_children_number)
            print actions
            raise NameError('Stopping program')
    #funcao atribui um no filho a um nó pai que ainda nao teve todos os filhos explorados
    def assign_child(self, identifier,actions):#ele so tera os ids, nao os obj em si
        if(self.firstTime):
            self.max_child_number = len(actions)
            self.firstTime = False
        self.know_children_number+=1
        self.children.append(identifier)
        if(self.know_children_number == self.max_child_number):
            self.expanded = True
            #print "expanded"
            #print self.max_child_number
    def set_id(self,identifier):
        self.identifier = identifier
    def add_value(self, value):
        self.total+=value/(69.0)#convertendo para intervalo de 0 a 1
        self.valueQ=self.total/self.count
    def add_count(self):
        self.count+=1   

In [11]:
class MCST:
    def __init__(self):
        self.nodes = {}
        self.trajectory = []
        self.identifier = -1#se lembre sempre de colocar o root
    def add_node(self,node):
        self.identifier+=1
        self.nodes[self.identifier] = node
        return self.identifier
    def add_trajectory(self,identifier):
        self.trajectory.append(identifier)
    def resetTrajectory(self):
        self.trajectory = []

In [12]:
class UCT:
    def __init__(self,c):
        self.c = c
        self.tree = MCST()
        
    def treeReset(self,rootNode):
        self.tree = MCST()
        self.tree.add_node(rootNode)
        
    def resetTrajectory(self):
        self.tree.resetTrajectory()
        
    def electUCT(self,node,criteria):
        elected_id = ""
        if(criteria == "Max"):
            maxValueQ = -99999999
            for child_id in node.children:
                currentVal = self.tree.nodes[child_id].valueQ
                if(currentVal>maxValueQ):
                    maxValueQ = currentVal
                    elected_id = child_id
        elif(criteria == "Robust"):
            maxCount = 0
            for child_id in node.children:
                currentCount = self.tree.nodes[child_id].count
                if(currentCount>maxCount):
                    maxCount = currentCount
                    elected_id = child_id
        return self.tree.nodes[elected_id]
            
            
    def calculateUCT(self,node,fatherNode):
        if(fatherNode.count!=0 and node.count!=0):
            return node.valueQ + 2*self.c*(math.sqrt(2*math.log(fatherNode.count)/node.count))
        else:#na conf atual nunca deveria acontecer
            return 9999999#infinidade, para nos nao explorados
        
    def selectUCT(self,currentNode):
        self.tree.add_trajectory(currentNode.identifier)#adicione o root na trajetoria
        return self.select(currentNode)
    def select(self,currentNode):
        if (not currentNode.children or not currentNode.expanded):
            return currentNode###########deveria adicionar trajetoria tbm?
        #o uct so e aplicado em nos com um numero de visitas T
        if(currentNode.count>=T):
            maxVal = -99999999 #tomar cuidado
            choosenNode = currentNode
            for child_id in currentNode.children:
                #print "node:"+str(node.player)
                currentVal = self.calculateUCT(self.tree.nodes[child_id],currentNode)
                if(currentVal>maxVal):
                    maxVal = currentVal
                    choosen_child_id = child_id #aki deve escolher o melhor
        else:#escolha com o msm metodo da simulacao
            playout_act = control.policyAct(state,currentNode.player)
            for child_id in currentNode.children:
                if(self.tree.nodes[child_id].act==playout_act):#tem como melhorar?
                    choosen_child_id = child_id
            
        self.tree.add_trajectory(choosen_child_id)
        choosenNode= self.select(self.tree.nodes[choosen_child_id])
        return choosenNode
    
    def replay(self,node,state,domino):
        history = self.tree.trajectory
        if not history:
            return state
        for h in history:
            node = self.tree.nodes[h]
            #print "player "+str(node.player)+" replayed "+str(node.act)
            domino.playAction(state,node.act,node.player)
            
        return state
        
    def expand(self,node,state,domino):
        if not node.isTerminal:    
            state = self.replay(node,state,domino)
            #a proxima jogada vai ser do oponente
            if(node.player == "rootP1" or node.player == "p2"):
                actions = domino.possibleActions(state,"p1")
                #print "state: "+str(state)
                choosen_act = node.choosen_act(actions)#pega 1 acao ainda nao explorada
                state= domino.playAction(state,choosen_act,"p1")
            else:
                actions = domino.possibleActions(state,"p2")
                choosen_act = node.choosen_act(actions)
                state= domino.playAction(state,choosen_act,"p2")
            #com mais jogadores teria tambem que respeitar a ordem sempre
            if(node.player == "p2" or node.player == "rootP1"):
                new_node = Node("p1",choosen_act)
            else:
                new_node = Node("p2",choosen_act)
            new_id = self.tree.add_node(new_node)
            new_node.set_id(new_id)
            node.assign_child(new_id,actions)
            self.tree.add_trajectory(new_id)#para backpropagation
            if(state[0]>=3):
                new_node.isTerminal = True
                rootNode = self.tree.nodes[0]
                if(rootNode.player == "rootP1"):# a recompensa ficara gravada no ultimo
                    new_node.reward = domino.reward(state,"p1")#no para evitar recalculo
                else:
                    new_node.reward = domino.reward(state,"p2")
            return new_node
        else:
            return node
    
    def playoutRL(self,lastNode,state,domino,control):
        if lastNode.isTerminal:
            return lastNode.reward
        if(lastNode.player=="p1"):
            player = "p2"
        else:
            player = "p1"
        act = control.policyAct(state,player)
        #print "Playout: "
        while(state[0]<3):
            state = domino.playAction(state,act,player)
            #print "   "+ player + " played "+str(act)
            if(player=="p1"):
                player = "p2"
            else:
                player = "p1"
            act = control.policyAct(state,player)
        #sempre retorne a recompensa q o p1 ganha, lembrando de ativar a punicao,
        #para que o retorno nao seja sempre 0(quando perder), mas sim a qtd negativa
        #de pontos ganhos pelo oponente
        rootNode = self.tree.nodes[0]
        if(rootNode.player == "rootP1"):
            return domino.reward(state,"p1")
        else:
            return domino.reward(state,"p2")
        
        
        
    def backpropagate(self,reward):
        history = self.tree.trajectory#se lembrar de tirar self.tree do global
        for h in history:
            node = self.tree.nodes[h]
            node.add_count()
            node.add_value(reward)
            
        
        self.tree.resetTrajectory()    

In [13]:
#Controlador para utilizar e treinar rede Neural(Features)
class Control:
    def __init__(self,e,features,domino,gamma,tdLambda,uct):
        self.features = features
        self.featuresVector = np.zeros(1)
        self.e = e
        self.domino = domino
        self.tdLambda = tdLambda
        self.eTraces = np.zeros(features.getFeaturesWeights().size)
        self.gamma = gamma
        self.uct = uct
        
        
    
    def getFeatures(self):
        return self.features
    
    def setFeatures(self,features):
        self.features = features
        
   
    def fillTraces(self):
        self.eTraces[self.featuresVector>0] +=1
    
    def generateVector(self,state,act,player):
        self.featuresVector = self.features.atomicFeatures(state,act,player)
        
   
    def qValue(self,state,act,player):
        featureVals = self.features.atomicFeatures(state,act,player)
        val = np.dot(self.features.getFeaturesWeights(), featureVals)    
        return val

    def eGreedyPicker(self,actions,state,player):
        rand = np.random.random_sample()
        bestValue = 0 #tomar cuidado aqui
        cond = False
        if(rand > self.e):
            for act in actions:  
                value = self.qValue(state,act,player)
                if(not cond):
                    bestValue = value
                    choosenAct = act
                    cond = True
                if(value> bestValue):
                    bestValue = value
                    choosenAct = act
        else:
            choosenAct = random.choice(actions)

        return choosenAct
    
    def minmaxPicker(self,actions,state,player):
        rand = np.random.random_sample()
        index_choosenAct=0
        i= 0
        maxOpValue = -999999999
        simulation_state = copy.deepcopy(state)
        if(player == "p1"):
            opPlayer = "p2"
        else:
            opPlayer = "p1"
        if(rand > self.e):
            for act in actions:  
                value = self.qValue(simulation_state,act,player)
                #if(value == 0):
                    #return random.choice(actions)
                simulation_state = dominoes.playAction(simulation_state,act,player)
                opActions = dominoes.possibleActions(simulation_state,opPlayer)
                for opAct in opActions:
                    opValue = self.qValue(simulation_state,opAct,opPlayer)
                    if(opValue == 0):
                        return random.choice(actions)
                    if(opValue>maxOpValue):
                        maxOpValue = opValue
                        index_choosenAct = i

                simulation_state = copy.deepcopy(state)
                i+=1
            return actions[index_choosenAct]
        else:
            choosenAct = random.choice(actions)
            return choosenAct

    def policyAct(self,state,player):
        actions = self.domino.possibleActions(state,player)
        #escolhe a partir dos valores na q-value com tecnica e-greedy
        if(actions[0] is None):
            return actions
        else:
            choosenAct = self.eGreedyPicker(actions,state,player)

        return choosenAct
    
    def minmaxAct(self,state,player):
        actions = self.domino.possibleActions(state,player)
        if(actions[0] is None):
            return actions
        else:
            choosenAct = self.minmaxPicker(actions,state,player)

        return choosenAct
    
    #deveria ser mcstSearch
    def mcstAct(self,state,player,N):
        if(player == "p1"):
            rootNode = Node("rootP1",None)
        else:
            rootNode = Node("rootP2",None)
        self.uct.treeReset(rootNode)
        
        for i in range(N):
            new_state = copy.deepcopy(state)
            selected_node = self.uct.selectUCT(rootNode)
            new_node = self.uct.expand(selected_node,new_state,dominoes)
            r = self.uct.playoutRL(new_node,new_state,dominoes,self)
            self.uct.backpropagate(r)
            uct.resetTrajectory() #praq?
            elect_node = self.uct.electUCT(rootNode,"Max")
            #print elect_node.act
        return elect_node.act 
    
    def pimcSearch(self,state,player,mcstN,pimcN):
        actions = dominoes.possibleActions(state,player)
        #print "Actions: "+str(actions)
        actionsValues = {}
        maxAct = ''
        for i in range(len(actions)):
            actionsValues[i] = 0
         
        
        for i in range(pimcN):
            op_hand = self.Sample(state,player)
            #print "Sampled hand:" +str(op_hand)
            node = self.PerfInfoValue(state,op_hand,player,mcstN)
            #print "Choosen node: "+str(node.act)+ " id: "+str(node.identifier)
            #identifier e a propria pos na lista de acoes possiveis
            actionsValues[int(node.identifier)-1] += node.valueQ #corrige pos com -1
            #print "Search returned value: "+str(node.valueQ)
            #print "has now value: "+str(actionsValues[int(node.identifier)-1])
        argmax  = actionsValues[0]
        maxAct = 0
        for act_index in actionsValues:
            if(actionsValues[act_index]>argmax):
                argmax = actionsValues[act_index]
                maxAct = act_index
        #print "Best Action is "+str(actions[maxAct] )+" with value: "+str(argmax)
        return actions[maxAct]   
    
    def Sample(self,state,player):
        if(player=="p1"):
            op_hand = state[2]
        else:
            op_hand = state[1]
        info_set = op_hand+state[8]
        generated_op_hand = []
        
        for i in range(len(op_hand)):
            random_index = randrange(0,len(info_set))
            generated_op_hand.append(info_set[random_index])
            del info_set[random_index]
        return generated_op_hand
        
        
        
    
    def PerfInfoValue(self,state,op_hand,player,N):
        if(player == "p1"):
            backup = state[2] #mao real e guardada
            state[2] = op_hand # substituida pela mao gerada aleatoriamente
        else:
            backup = state[1] 
            state[1] = op_hand
        node = self.mcstAct(state,player,N) #busca no estado ja conhecido-perfeita informacao
        if(player == "p1"):
            state[2] = backup #retorna a imperfeita informacao
        else:
            state[1] = backup
        return node
    
    def alpha(self,vector):#calcua taxa de aprendizado de acordo com quantidade de features atuais
        
        unique, counts = np.unique(vector, return_counts=True)
        if((counts[0])==self.featuresVector.size):
            return 0.01
        a = 0.01/(counts[1]) #constante por numero total de 1
        return a
    
    def decayTraces(self):
        if(self.tdLambda>0):
            feat = self.featuresVector
            self.eTraces[feat>0]=self.eTraces[feat>0]*self.gamma*self.tdLambda
            self.auxFeaturesVector = self.eTraces
            self.auxFeaturesVector[self.featuresVector==0] = 0#retorna trace das carac atuais
            return self.auxFeaturesVector 
        else:
            return self.featuresVector
        
    
    def evaluate(self,state,val,act,player,actType):
        self.fillTraces()
        r = dominoes.reward(state,player)
        if(actType=="mcst"):
            next_action = self.mcstAct(state,player,20)
        elif(actType=="minmax"):
            next_action = self.minmaxAct(state,player)
        else:
            next_action = self.policyAct(state,player)
        new_value = self.qValue(state,next_action,player)
        target = r+ self.gamma * new_value        # gamma = 0.9
        delta = target - val # prediction error
        featureWeights = self.features.getFeaturesWeights()
        featureWeights += self.alpha(self.featuresVector) * delta *self.decayTraces()
        self.features.setFeaturesWeights(featureWeights)
    
        elements = [state,new_value,next_action,r]
        return elements

In [ ]:
#Complexity test2 on number of actions
def minimaxAct(state,player):
    copy_state = copy.deepcopy(state)
    count = 0
    actions = dominoes.possibleActions(state,player)
    #print actions
    for act in actions:
        count+=1
        copy_state = copy.deepcopy(state)
        count = minmax(copy_state,act,"p1",count)
    print count
    
def minmax(state,act,player,count):
    #print "Playing: "+str(act)
    state = dominoes.playAction(state,act,player)
    if(state[0]==3):
        return count
    if(player == "p1"):
        player = "p2"
    else:
        player = "p1"   
    actions = dominoes.possibleActions(state,player)
    #print "possibel actions: "+str(actions)
    for act in actions:
        count+=1
        copy_state = copy.deepcopy(state)
        count = minmax(copy_state,act,player,count)
    return count
        
dominoes = Domino(0,0,0,0,True,"Draw")
state = [1,[(2,4),(3,5),(1,1),(1,4),(0,4),(1,2),(4,4)],
         [(3,0),(1,6),(1,3),(3,4),(5,4),(2,2),(3,6)],
         [],-1,-1,False,False,[(6,6),(4,6),(3,3),(2,0),(2,3),(2,5),(2,6)]]

minimaxAct(state,"p1")

In [16]:
#Training1 : Treinar extensive network
dominoes = Domino(0,0,0,0,True,"Block")#
e = 0.03  # epsilon-greedy proportion
featureWeights1 = np.zeros(110)#630 comb 37820 comb2
featureWeights2  = np.zeros(110)

network1 = ExtensiveNetwork()
network2 = ExtensiveNetwork()
features1 = Features(featureWeights1,network1)
features2 = Features(featureWeights2,network2)
uct = UCT(0.1)
T = 0 
controlP1 = Control(0.03,features1,dominoes,0.4,0,uct)
controlP2 = Control(0.03,features2,dominoes,0.4,0,uct)

r = 0

duration = 0
#self-play
firstplay = 0
accumulatedRewardP1 = 0
accumulatedRewardP2 = 0
P1Wins = 0
P2Wins = 0

#para medir tempo
#start_time = timeit.default_timer()
#print(timeit.default_timer() - start_time)
for i in range(0,10000):
    flag = True
    state = dominoes.startGame()

    actP1 = controlP1.minmaxAct(state,"p1")
    val1 = controlP1.qValue(state,actP1,"p1")
    actP2 = controlP2.policyAct(state,"p2")
    val2 = controlP2.qValue(state,actP2,"p2")
    controlP2.generateVector(state,actP2,"p2")
    while(dominoes.termination(state) == False):
        
        controlP1.generateVector(state,actP1,"p1") 
        state = dominoes.playAction(state, actP1,"p1")
        state,val2,actP2,r =controlP2.evaluate(state,val2,actP2,"p2","default")
        
        if(state[0]>=3):
            flag = False
        controlP2.generateVector(state,actP2,"p2")
        accumulatedRewardP2+=r
        state = dominoes.playAction(state, actP2,"p2")
        state,val1,actP1,r =controlP1.evaluate(state,val1,actP1,"p1","minmax")
        accumulatedRewardP1 +=r
        if(dominoes.termination(state) == True):
            if(flag):
                state,val2,actP2,r =controlP2.evaluate(state,val2,actP2,"p2","default")
                accumulatedRewardP2+=r
            if(state[0]==4):
                P1Wins+=1
            else:
                P2Wins+=1
        
featureWeights1 = controlP1.getFeatures().getFeaturesWeights()
featureWeights2 = controlP2.getFeatures().getFeaturesWeights()

print "Resulting weight1: "+str(featureWeights1)
#print "Resulting weight2: "+str(featureWeights2)
print "P1victories: "+str(P1Wins) + " P1 Accumulated reward: "+ str(accumulatedRewardP1)
print "P2victories: "+str(P2Wins) + " P2 Accumulated reward: "+ str(accumulatedRewardP2)
if(P1Wins>P2Wins):
    winnerWeigth = featureWeights1
else:
    winnerWeigth = featureWeights2
np.save('ExtensiveNetMinMax', winnerWeigth)

Resulting weight1: [ -3.06871536e-02   9.31693139e-02  -2.96977463e-01  -2.80706560e-01
  -6.50085005e-01  -7.42498397e-01  -8.41546793e-01  -1.24227221e+00
   5.71065832e-02   5.73049591e+00  -6.52594498e-01  -2.29576140e+00
  -1.84812975e+00  -5.13398712e-01   1.54015280e-02  -5.91819533e+00
  -7.79631883e-01   1.97342521e+00   2.51374566e+00   2.02581787e+00
   1.54015280e-02   1.68898790e-02   8.05816643e-02  -3.06871536e-02
   9.29742602e-02   1.01278338e-01   9.78386536e-02   3.21303015e-02
  -1.84235676e-03   1.44321036e-01  -3.06871536e-02   9.29742602e-02
   1.01278338e-01   9.78386536e-02   3.21303015e-02  -1.84235676e-03
   1.44321036e-01   1.46190890e-01   6.76643458e-02   5.45341435e-01
  -3.09820555e-02   7.00336843e-01   7.47166833e-01   7.48741214e-01
   6.96895165e-01   4.09275740e-01   2.54004017e-02  -1.05418139e+00
  -1.56573045e+00  -1.00010589e+00   0.00000000e+00   7.97727961e-01
  -3.71157352e-02  -2.51051205e-01  -6.96080530e-02  -3.93988860e-03
   0.00000000e+

In [ ]:
#Automatic Training1 : Treinar extensive network
for l in range (0,11):
    dominoes = Domino(0,0,0,0,True,"Block")#
    e = 0.09  # epsilon-greedy proportion
    featureWeights1 = np.zeros(110)#630 comb 37820 comb2
    featureWeights2 = np.zeros(110)
    network1 = ExtensiveNetwork2()
    network2 = ExtensiveNetwork2()
    features1 = Features(featureWeights1,network1)
    features2 = Features(featureWeights2,network2)
    uct = UCT(0.1)
    T = 0 
    controlP2 = Control(0.03,features1,dominoes,0.4,l/10,uct)
    controlP1 = Control(0.03,features2,dominoes,0.4,l/10,uct)
    r = 0

    duration = 0
    #self-play
    firstplay = 0
    accumulatedRewardP1 = 0
    accumulatedRewardP2 = 0
    P1Wins = 0
    P2Wins = 0


    for i in range(0,1000):
        flag = True
        state = dominoes.startGame()

        actP1 = controlP1.policyAct(state,"p1")
        val1 = controlP1.qValue(state,actP1,"p1")
        actP2 = controlP2.policyAct(state,"p2")
        val2 = controlP2.qValue(state,actP2,"p2")
        controlP2.generateVector(state,actP2,"p2")
        while(dominoes.termination(state) == False):

            controlP1.generateVector(state,actP1,"p1") 
            state = dominoes.playAction(state, actP1,"p1")
            state,val2,actP2,r =controlP2.evaluate(state,val2,actP2,"p2","default")

            if(state[0]>=3):
                flag = False
            controlP2.generateVector(state,actP2,"p2")
            accumulatedRewardP2+=r
            state = dominoes.playAction(state, actP2,"p2")
            state,val1,actP1,r =controlP1.evaluate(state,val1,actP1,"p1","default")
            accumulatedRewardP1 +=r
            if(dominoes.termination(state) == True):
                if(flag):
                    state,val2,actP2,r =controlP2.evaluate(state,val2,actP2,"p2","default")
                    accumulatedRewardP2+=r
                if(state[0]==4):
                    P1Wins+=1
                else:
                    P2Wins+=1

    featureWeights1 = controlP1.getFeatures().getFeaturesWeights()
    featureWeights2 = controlP2.getFeatures().getFeaturesWeights()

    print "Resulting weight1: "+str(featureWeights1)
    #print "Resulting weight2: "+str(featureWeights2)
    print "P1victories: "+str(P1Wins) + " P1 Accumulated reward: "+ str(accumulatedRewardP1)
    print "P2victories: "+str(P2Wins) + " P2 Accumulated reward: "+ str(accumulatedRewardP2)
    if(P1Wins>P2Wins):
        winnerWeigth = featureWeights1
    else:
        winnerWeigth = featureWeights2
    name = "Training"+str(l)
    np.save(name, winnerWeigth)
    print "Trained weigth: "+str(l)

In [ ]:
#TD Leaf Training
dominoes = Domino(0,0,0,0,True,"Block")#winReward,lostReward,blockingReward,blockedReward,lostPunition
e = 0.03  # epsilon-greedy proportion
featureWeights1 = np.zeros(300)
featureWeights2 = np.zeros(300)
featureWeights3 = np.load('neuralNetP1.npy')
extNetwork = ExtensiveNetwork()
network= Network()
features1 = Features(featureWeights1,extNetwork)
features2 = Features(featureWeights2,extNetwork)
features3 = Features(featureWeights3,network)
uct = UCT(0.1)
T = 0 
controlP1 = Control(0.03,features1,dominoes,0.4,0,uct)
controlP2 = Control(0.03,features2,dominoes,0.4,0,uct)
r = 0

duration = 0
#self-play
firstplay = 0
accumulatedRewardP1 = 0
accumulatedRewardP2 = 0
P1Wins = 0
P2Wins = 0

for i in range(0,1000):
    flag = True
    state = dominoes.startGame()
    controlP1.setFeatures(features3)
    controlP2.setFeatures(features3)
    actP1 = controlP1.mcstAct(state,"p1",20)
    val1 = controlP1.qValue(state,actP1,"p1")
    actP2 = controlP2.mcstAct(state,"p2",20)
    val2 = controlP2.qValue(state,actP2,"p2")
    controlP1.setFeatures(features1)
    controlP2.setFeatures(features2)
    controlP2.generateVector(state,actP2,"p2")
    while(dominoes.termination(state) == False):
        
        controlP1.generateVector(state,actP1,"p1")
        state = dominoes.playAction(state, actP1,"p1")
        state,val2,actP2,r =controlP2.evaluate(state,val2,actP2,"p2","TdLeaf",features3)
        if(state[0]>=3):
            flag = False
        controlP2.generateVector(state,actP2,"p2")
        accumulatedRewardP2+=r
        state = dominoes.playAction(state, actP2,"p2")
        state,val1,actP1,r =controlP1.evaluate(state,val1,actP1,"p1","TdLeaf",features3)
        accumulatedRewardP1 +=r
        
        if(dominoes.termination(state) == True):
            if(flag):
                state,val2,actP2,r =controlP2.evaluate(state,val2,actP2,"p2","TdLeaf",features3)
                accumulatedRewardP2+=r
            if(state[0]==4):
                P1Wins+=1
            else:
                P2Wins+=1
        
featureWeights1 = controlP1.getFeatures().getFeaturesWeights()
featureWeights2 = controlP2.getFeatures().getFeaturesWeights()

print "Resulting weight1: "+str(featureWeights1)
#print "Resulting weight2: "+str(featureWeights2)
print "P1victories: "+str(P1Wins) + " P1 Accumulated reward: "+ str(accumulatedRewardP1)
print "P2victories: "+str(P2Wins) + " P2 Accumulated reward: "+ str(accumulatedRewardP2)
if(P1Wins>P2Wins):
    winnerWeigth = featureWeights1
else:
    winnerWeigth = featureWeights2
np.save('ExtensiveNetTDLeaf', winnerWeigth)

In [ ]:
#Training2 : Treinar so como 1 jogador e o suficiente para cobrir todos os casos importantes
dominoes = Domino(0,0,0,0,True,"Block")#winReward,lostReward,blockingReward,blockedReward,lostPunition
e = 0.03  # epsilon-greedy proportion
featureWeights1 = np.zeros(35)#630 comb 37820 comb2
featureWeights2 = np.load('neuralNetP1.npy')
network1 = Network()
network2 = Network()
features1 = Features(featureWeights1,network1)
features2 = Features(featureWeights2,network2)
uct = UCT(0.1)
T = 0 trol(0.03,features1,dominoes,0.4,0,uct)
controlP2 = C
controlP1 = Conontrol(0.03,features2,dominoes,0.4,0,uct)
r = 0

duration = 0
#self-play
firstplay = 0
accumulatedRewardP1 = 0
accumulatedRewardP2 = 0
P1Wins = 0
P2Wins = 0

for i in range(0,1000):
    flag = True
    state = dominoes.startGame()
    actP1 = controlP1.mcstAct(state,"p1",20)
    val1 = controlP1.qValue(state,actP1,"p1")
    actP2 = controlP2.mcstAct(state,"p2",20)
    val2 = controlP2.qValue(state,actP2,"p2")
    controlP2.generateVector(state,actP2,"p2")
    while(dominoes.termination(state) == False):
        
        controlP1.generateVector(state,actP1,"p1")
        state = dominoes.playGame(state, actP1,"p1")
        state,val2,actP2,r =controlP2.evaluate(state,val2,actP2,"p2")
        if(state[0]>=3):
            flag = False
        controlP2.generateVector(state,actP2,"p2")
        accumulatedRewardP2+=r
        state = dominoes.playGame(state, actP2,"p2")
        state,val1,actP1,r =controlP1.evaluate(state,val1,actP1,"p1")
        accumulatedRewardP1 +=r
        
        if(dominoes.termination(state) == True):
            if(flag):
                state,val2,actP2,r =controlP2.evaluate(state,val2,actP2,"p2")
                accumulatedRewardP2+=r
            if(state[0]==4):
                P1Wins+=1
            else:
                P2Wins+=1
        
featureWeights1 = controlP1.getFeatures().getFeaturesWeights()
featureWeights2 = controlP2.getFeatures().getFeaturesWeights()

print "Resulting weight1: "+str(featureWeights1)
#print "Resulting weight2: "+str(featureWeights2)
print "P1victories: "+str(P1Wins) + " P1 Accumulated reward: "+ str(accumulatedRewardP1)
print "P2victories: "+str(P2Wins) + " P2 Accumulated reward: "+ str(accumulatedRewardP2)
if(P1Wins>P2Wins):
    winnerWeigth = featureWeights1
else:
    winnerWeigth = featureWeights2
np.save('MCST_neuralNetP1', winnerWeigth)

In [ ]:
#initial2
dominoes = Domino(0,0,0,0,True)#tem que saber diferenciar um outcome ruim pra um 0
state = dominoes.startGame()
uct = UCT(0.1)
state = [1,[(3,0),(1,6),(1,3)],[(2,4),(3,5),(1,1)],[],-1,-1,False,False,
        [(0,0),(2,3),(4,4)]]
e = 0.000001
T = 0 #threshold, num de visitas necessarios para se aplicar o uct na selecao
featureWeights1 = np.load('neuralNetP1.npy')
featureWeights2 = np.load('neuralNetP1.npy')
network1 = Network()
network2 = Network()
features1 = Features(featureWeights1,network1)
features2 = Features(featureWeights2,network2)
controlP1 = Control(0.03,features2,dominoes,0.4,0,uct)
controlP2 = Control(0.03,features2,dominoes,0.4,0,uct)
tree = MCST()
player = "rootP1"
rootNode = Node(player,None)#sempre sera um no fantasma para as acoes possiveis
new_state = copy.deepcopy(state)
tree.add_node(rootNode)#nunca se esqueca de adicionar o root na arvore antes

In [34]:
#IA vs IA
dominoes = Domino(0,0,0,0,True,"Block")#
e = 0.000001  # aqui n ha aprendizagem
r = 0
duration = 0
P1Wins = 0
P2Wins = 0
firstplay =1
accumulatedRewardP1 = 0
accumulatedRewardP2 = 0
count = 0
featureWeights1 = np.load('ExtensiveNetDefault.npy')
featureWeights2 = np.load('ExtensiveNetMinMax.npy')
network1 = ExtensiveNetwork()
network2 = ExtensiveNetwork()
features1 = Features(featureWeights1,network1)
features2 = Features(featureWeights2,network2)
uct = UCT(0.1)
T = 0 
controlP1 = Control(0.03,features1,dominoes,0.4,0,uct)
controlP2 = Control(0.03,features2,dominoes,0.4,0,uct)
r = 0
for i in range(0,1000):
    
    state = dominoes.startGame()
    firstplay = random.randint(0,1)
    #firstplay = 1
    while(dominoes.termination(state) == False):
        if(firstplay==0):
            
            actP1 = controlP1.policyAct(state,"p1")
            dominoes.playAction(state, actP1,"p1")
            actP2 = controlP2.minmaxAct(state,"p2")
            dominoes.playAction(state, actP2,"p2")
        else:
            actP2 = controlP2.minmaxAct(state,"p2")
            dominoes.playAction(state, actP2,"p2")
            actP1 = controlP1.policyAct(state,"p1")
            dominoes.playAction(state, actP1,"p1")
        accumulatedRewardP1 += dominoes.reward(state,"p1")
        accumulatedRewardP2 +=dominoes.reward(state,"p2")
    if(state[0]==4):
        P1Wins+=1
    else:
        P2Wins+=1
print "Player1 winnings: "+str(P1Wins) + " Player2 winnings: "+str(P2Wins)
print "Player1 rewards: "+str(accumulatedRewardP1)+" Player2 rewards: "+str(accumulatedRewardP2)

Player1 winnings: 651 Player2 winnings: 349
Player1 rewards: 4075 Player2 rewards: -4075


In [ ]:
#IA vs Other

def takeout(state,takeout_piece):
    i = 0
    for bone in state[8]:
        #print "bone: "+str(bone) + " takeout: "+str(takeout_piece)
        if(bone==takeout_piece):
            del state[8][i]
        i+=1
    return state

def takeoutHand(state,takeout_piece):
    cond = False
    i = 0
    for bone in state[2]:
        if(bone==takeout_piece):
            print "Deleted bone: "+str(bone)
            del state[8][i]
            cond = True
        i+=1
    return [state,cond]
      
    
def iaPlay(state,player):
    print("inform what side(left/right) the piece has matched or None ")
    side = raw_input("")
    if(side == "None"):
        dominoes.playGame(state, None,player)
        return
    print "Inform Left and Right:"
    bone = input("")
    state = takeout(state,bone)
    state,cond = takeoutHand(state,bone)
    state[2].insert(0,bone)#coloca para se ter a acao
    actions = dominoes.possibleActions(state,player)
    if(len(actions)>1 and actions[1][1]==side):# no caso de ter 2 acoes da msm peca
        dominoes.playGame(state, actions[1],player)
    else:#caso 1 so acao ou caso da 1 rodada(valor,valor,index)
        dominoes.playGame(state, actions[0],player)
    del state[2][0] #retira
    if(not cond): 
        if(len(state[2])>0):
            played_piece = state[2][0] # retira-se uma peca da mao do op para equilibrar
            del state[2][0]
            state[8].insert(0,played_piece)
        
    print "Sides after p1 play: "+str(state[4]) + ","+str(state[5])
     
        
cr1 = 0
r1 = 0
cr2 = 0
r2 = 0
duration = 0
count = 0
P1Wins = 0
firstplay = 1
count+=1
r1 = 0
r2 = 0
state = dominoes.startGame()
#firstplay = random.randint(0,1)
firstplay = 0
while(dominoes.termination(state) == False):
    state[1] = []
    state[2] = []
    op_hand = []
    state[8] = []
    state[8] = [(x,y) for x in range(7) for y in range(x,7)]
    print "Inform my hand"
    for i in range(7):
        print "Piece"+str(i)
        print "Inform Left and Right:"
        bone = input("")
        state[1].append(bone)
        state = takeout(state,bone)
        
    print "state now: "+str(state)    
    boneyard = state[8] 
    op_hand,boneyard = dominoes.buy(op_hand, 7, boneyard)
    state[2] = op_hand
    state[8] = boneyard
    print state
    if(firstplay==0):
        iaPlay(state,"p2")
        print "my hand: "+str(state[2])
        actP1 = controlP1.pimcSearch(state,"p1",20,100)
        dominoes.playGame(state, actions[0],player)
    else:
        print "my hand: "+str(state[2])
        actP1 = controlP1.pimcSearch(state,"p1",20,100)
        dominoes.playGame(state, actions[0],player) 
        iaPlay(state,"p2")
    r1 += dominoes.reward(state,"p1")
    r2 += dominoes.reward(state,"p2")

    #print "State after both plays: "+str(state)
    print "field after both plays: "+str(state[3])
if(state[0]==4):
    print "IA Wins"
else:
    print "I Win"

print "My Game reward: "+str(r2)
print "IA Game reward: "+str(r1)
cr1 +=r1
cr2 +=r2
print "IA Cumulative reward: "+str(cr1)
print "My Cumulative reward: "+str(cr2)
print state
print "END OF GAME "+str(count)
print "XXXXXXXXXXXXXXXXXXXXXXXXXX"


print "Player1 winnings: "+str(P1Wins)

In [ ]:
#Draw and Block Tester
dominoes = Domino(0,0,0,0,True,"Draw")
state = [1,[(6,6),(0,3),(0,0)],[(0,2),(2,2),(3,3)],[],-1,-1,False,False,[(6,5),(1,4),(2,4)]]
actions = dominoes.possibleActions(state,"p1")
print state,actions
print "P1 plays 6,6 from its hand"
dominoes.playAction(state,actions[0],"p1")
actions = dominoes.possibleActions(state,"p2")
print state,actions
print "P2 plays 6,5 from boneyard"
dominoes.playAction(state,actions[0],"p2")
actions = dominoes.possibleActions(state,"p1")
print state,actions
print "P1 does not have playable pieces on hand and boneyard, take all boneyard pieces, and is blocked"
dominoes.playAction(state,actions[0],"p1")
print state,actions
print "P2 too does not have playable pieces. Both blocked and GameOver"
actions = dominoes.possibleActions(state,"p2")
dominoes.playAction(state,actions[0],"p2")
print state,actions
print "P2 wins! (lower score than P1)"
r1 = dominoes.reward(state,"p1")
r2 = dominoes.reward(state,"p2")
print "P1: "+str(r1) + " P2: "+str(r2)
print state

In [ ]:
#pre-load mcst
dominoes = Domino(0,0,0,0,False)#lost_punition para recomp de vit dos nós op serem negativos 
state = dominoes.startGame()
featureWeights = np.load('neuralNetP1.npy')
network = Network2()
features = Features(featureWeights,network)
uct = UCT(0.1)
controlP1 = Control(0.03,features,dominoes,0.4,0,uct)
tree = MCST()
print state

T = 0 #threshold, num de visitas necessarios para se aplicar o uct na selecao
player = "rootP2"
rootNode = Node(player,None)#sempre sera um no fantasma para as acoes possiveis
new_state = copy.deepcopy(state)
tree.add_node(rootNode)#nunca se esqueca de adicionar o root na arvore antes
print featureWeights

In [ ]:
#pre-load normal
dominoes = Domino(0,0,0,0,False)#lost_punition para recomp de vit dos nós op serem negativos 
state = dominoes.startGame()
uct = UCT(0.1)
#rootNode = Node("rootP2",None)
#tree.add_node(rootNode)
T = 0 #threshold, num de visitas necessarios para se aplicar o uct na selecao
featureWeights1 = np.load('neuralNetP1.npy')
featureWeights2 = np.load('neuralNetP1.npy')
network1 = Network35()
network2 = Network2()
combNetwork = CombinationalNetwork(35,network2,"Pairwise")
features1 = Features(featureWeights1,combNetwork)
features2 = Features(featureWeights2,network1)
controlP1 = Control(0.03,features2,dominoes,0.4,0,uct)
controlP2 = Control(0.03,features2,dominoes,0.4,0,uct)
 

In [ ]:
#ia vs human
def iaPlay(state,player,flag):
    actP1 = control.policyAct(state,player)
    dominoes.playGame(state, actP1,player)
    if(actP1 is None):
        print "IA Blocked"
    elif(actP1[0] is not None and flag == False):
        print "IA Played: "+ str(actP1[0])+ " on the " + str(actP1[1])
    elif(actP1[0] is None):
        print "IA Blocked"
    else:
        print "IA Played: "+ str(actP1[0])+" left and "+ str(actP1[1])+ " right"
        flag = True
        
        #print "State after p1 play: "+str(state)
    print "Sides after p1 play: "+str(state[4]) + ","+str(state[5])

def myPlay(state,player,flag):
    
    actionss2 = dominoes.possibleActions(state,player)
    if(actionss2[0] is None):
        print("I'm Blocked!!")
        actP2 = control.policyAct(state,player)
        dominoes.playGame(state, actP2,player)
    else:
        print("Choose one action: ")
        k = 0
        if(flag):
            for act in actionss2:
                print str(k)+": "+str(act[0])+ " on the left and "+str(act[1])+" on the right"
                k+=1
        else:
            for act in actionss2:
                print str(k)+": " + str(act[0])+ " on the "+str(act[1])
                k+=1
        actionPos=input("")
        dominoes.playGame(state, actionss2[actionPos],player)

cr1 = 0
r1 = 0
cr2 = 0
r2 = 0
duration = 0
count = 0
P1Wins = 0
firstplay = 1
while(cr1<50 and cr2<50):
    count+=1
    r1 = 0
    r2 = 0
    state = dominoes.startGame()
    firstplay = random.randint(0,1)
    flag = True
    while(dominoes.termination(state) == False):
        if(firstplay==0):
            iaPlay(state,"p1",flag)
            flag = False
            print "my hand: "+str(state[2])
            myPlay(state,"p2",flag)
        else:
            print "my hand: "+str(state[2])
            myPlay(state,"p2",flag)
            flag = False
            iaPlay(state,"p1",flag)
        r1 += dominoes.reward(state,"p1")
        r2 += dominoes.reward(state,"p2")
        
        #print "State after both plays: "+str(state)
        print "field after both plays: "+str(state[3])
    if(state[0]==4):
        print "IA Wins"
    else:
        print "I Win"
        
    print "My Game reward: "+str(r2)
    print "IA Game reward: "+str(r1)
    cr1 +=r1
    cr2 +=r2
    print "IA Cumulative reward: "+str(cr1)
    print "My Cumulative reward: "+str(cr2)
    print state
    print "END OF GAME "+str(count)
    print "XXXXXXXXXXXXXXXXXXXXXXXXXX"
    
    
print "Player1 winnings: "+str(P1Wins)

In [ ]:
#Complexity test1 on number of actions
def minimaxAct(state,player):
    copy_state = copy.deepcopy(state)
    count = 0
    actions = dominoes.possibleActions(state,player)
    #print actions
    for act in actions:
        count+=1
        copy_state = copy.deepcopy(state)
        count = minmax(copy_state,act,"p1",count)
    print count
    
def minmax(state,act,player,count):
    #print "Playing: "+str(act)
    state = dominoes.playAction(state,act,player)
    if(state[0]==3):
        return count
    if(player == "p1"):
        player = "p2"
    else:
        player = "p1"   
    actions = dominoes.possibleActions(state,player)
    for act in actions:
        count+=1
        copy_state = copy.deepcopy(state)
        count = minmax(copy_state,act,player,count)
    return count
        
dominoes = Domino(0,0,0,0,True,"Draw")
state = [1,[(3,0),(1,6),(1,3)],
         [(2,4),(3,5),(1,1)],[],-1,-1,False,False,[(1,4),(3,4)]]
minimaxAct(state,"p1")

In [ ]:
#manual state for test
state = [1,[(2,4),(0,3),(0,0),(3,6),(0,1),(0,6),(5,5)],[(2,2),(6,6),(1,5),(1,4),(1,6),(0,2),(1,3)],[],-1,-1,False,False]
new_state = copy.deepcopy(state)

In [ ]:
#manual state2 for test
#Aqui é testado o uso do mcst num jogo já em percurso
state = [1,[(2,4),(0,3),(0,0)],[(0,2),(2,2),(3,3)],[],-1,-1,False,False]
actions = dominoes.possibleActions(state,"p1")
dominoes.playGame(state, actions[0],"p1")#jogue 0,0
#actions = dominoes.possibleActions(state,"p2")#jogue 2,0
#dominoes.playGame(state, actions[0],"p2")
#resultados previstos: escolhe 2,4 como proxima acao, pois tem maior
#recompensa(10), evitando 0,3 que gera 2 caminhos de recompensa 4 e 6

In [ ]:
#manual state3 for test
#o branch do 2,4 logo domina com rec 0.2, mas ele n deixa de tentar o branch 3,5
# que tem possíveis recompensas 0.18 e 0.09 . Branch 1,1 so tem 0.09 como rec
state = [1,[(2,4),(3,5),(1,1)],[(3,0),(1,6),(1,3)],[],-1,-1,False,False]
new_state = copy.deepcopy(state)

In [ ]:
#manual state4 for test
#ultimate minimal case, decision between victory and defeat
state = [1,[(0,4),(2,2)],[(1,3)],[],-1,-1,False,False]
new_state = copy.deepcopy(state)

In [ ]:
#test run
tree.resetTrajectory()
for i in range(20):
    new_state = copy.deepcopy(state)
    selected_node = uct.selectUCT(rootNode)
    print "##############################################################################"
    print "node selected on iteration"+str(i)+": "
    print "     player: "+str(selected_node.player)+" and act: " +str(selected_node.act)
    new_node = uct.expand(selected_node,new_state,dominoes)
    print "new expansion player: "+str(new_node.player)+" and act: "+str(new_node.act)
    r = uct.playoutRL(new_node,new_state,dominoes)
    print "reward on iteration"+str(i)+": "+str(r)
    uct.backpropagate(r)
    tree.resetTrajectory()
    print "selected node now has value: "+str(selected_node.valueQ)
print "***********************************************************************"
elected_node = uct.electUCT(rootNode,"Max")
#deve retornar o 1 no nao expandido, o 1 filho no caso
print "after exhaustive search, elect node is: "
print "Value: "+str(elected_node.valueQ)+" and act: " +str(elected_node.act)

In [ ]:
#Testando a expansão do root, ate a seleção do 1 filho dele
new_node,status = uct.expand(rootNode,new_state,dominoes)
print str(new_node.player)+ " act "+str(new_node.act)#1 filho
selected_node = uct.selectUCT(rootNode)
tree.resetTrajectory()
print str(selected_node.player)#deve retornar o 1 no nao expandido, o root no caso
for i in range(6):
    new_state = copy.deepcopy(state)
    selected_node = uct.selectUCT(rootNode)
    print "node selected on iteration"+str(i)+": "
    print "player: "+str(selected_node.player)+" and act: " +str(selected_node.act)
    new_node,status = uct.expand(selected_node,new_state,dominoes)
    print "new expansion player: "+str(new_node.player)+" and act: "+str(new_node.act)
    r = uct.playoutRL(new_node,new_state,dominoes)
    print "reward on iteration"+str(i)+": "+str(r)
    uct.backpropagate(r)
    tree.resetTrajectory()
    print "root now has value: "+str(rootNode.valueQ)

selected_node = uct.selectUCT(rootNode)
#deve retornar o 1 no nao expandido, o 1 filho no caso
print "after all childs expanded, new selected node is: "
print "player: "+str(selected_node.player)+" and act: " +str(selected_node.act)

In [ ]:
#numero de combinacoes
count = 0
atomFeatures = np.zeros(110)
for i in range(0,atomFeatures.size):
    for j in range (i,atomFeatures.size):
        for z in range (j,atomFeatures.size):
            count+=1
print count